In [9]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras


print(tf.__version__)
print(pd.__version__)
print(keras.__version__)

#which platform right now I located 
# import platform 
# print(platform.system())
# print(platform.release())
# print(platform.version())

2.4.0-rc0
1.2.3
2.4.0


In [7]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a*b

tf.print(c)

30
